In [1]:
# from maincode import Connect4Board
from random import randint

In [19]:
from maincode import Connect4Board

In [16]:
# class Connect4Board:

#     def __init__(self):
#         self.board = [[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]
#         # self.board = ["000000000000000000000000000000"]
#         self.highest =  [0,0,0,0,0]
    
#     def move(self, col, player):
#         if self.highest[col]>=6:
#             return 1
#         else:
#             self.board[5-self.highest[col]][col]=player
#             self.highest[col]+=1
#         return 0
    
#     def isWin(self, col, player):
#         row = 5-self.highest[col]+1

#         # principal diag
#         together=1
#         if col>0:
#             i=1
#             while((i<=col and i<=row) and self.board[row-i][col-i]==player):
#                 together+=1
#                 i+=1
#         if col<4:
#             i=1
#             while((col+i<=4 and row+i<=5) and self.board[row+i][col+i]==player):
#                 together+=1
#                 i+=1
#         if together>=4:
#             return True

#         # other diagonal
#         together=1
#         if col>0:
#             i=1
#             while((i<=col and row+i<=5) and self.board[row+i][col-i]==player):
#                 together+=1
#                 i+=1
#         if col<4:
#             i=1
#             while((col+i<=4 and i<=row) and self.board[row-i][col+i]==player):
#                 together+=1
#                 i+=1
#         if together>=4:
#             return True

#         # horizontal
#         together=1
#         if col>0:
#             i=1
#             while(i<=col and self.board[row][col-i]==player):
#                 together+=1
#                 i+=1
#         if col<4:
#             i=1
#             while(col+i<=4 and self.board[row][col+i]==player):
#                 together+=1
#                 i+=1
#         if together>=4:
#             return True

#         # vertical
#         together=1
#         if row >=3:
#             i=1
#             while(row+i<=5 and self.board[row+i][col]==player):
#                 together+=1
#                 i+=1
#         if together==4:
#             return True
        
#         return False

In [17]:
# MAIN
gboard = Connect4Board()
turn = 1
print(gboard.board)
# for i in range(6):
#     print(gboard.board[i])
while min(gboard.highest)<6:
    c = randint(0,4)
    while(gboard.move(col=c, player=turn)==1):
        c = randint(0,4)
    print("col:", c, " player:", turn)
    for i in range(6):
        print(gboard.board[i])
    if gboard.isWin(col=c, player=turn):
        print("Player ", turn, "has won")
        break
    if(turn==1):
        turn=2
    else:
        turn=1
    print()
    # print(min(gboard.highest))
        

[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]
col: 2  player: 1
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 1, 0, 0]

col: 3  player: 2
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 1, 2, 0]

col: 0  player: 1
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[1, 0, 1, 2, 0]

col: 4  player: 2
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[1, 0, 1, 2, 2]

col: 3  player: 1
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 1, 0]
[1, 0, 1, 2, 2]

col: 0  player: 2
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[2, 0, 0, 1, 0]
[1, 0, 1, 2, 2]

col: 1  player: 1
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[2, 0, 0, 1, 0]
[1, 1, 1, 2, 2]

col: 1  player: 2
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[2, 2, 0, 

In [ ]:
newboard = [[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]
print(newboard)

print(len(newboard))
for i in range(6):
    print (newboard[i])

In [24]:
def move(state, col, player):
        if state[col]!='0':
            print("here")
            return state
        else:
            for i in range(5,-1,-1):
                print(i)
                if state[5*i + col]=='0':
                    state[5*i + col]=chr(player+ord('0'))
                    break
        return state

In [25]:
cs = ['0']*30
csn = move(cs, 2, 1)

print(csn)

5
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0']
